In [1]:
import sys
sys.path.append('../src')

import re

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *


qtrs = {1: 'first', 2: 'second', 3: 'third', 4: 'fourth'}

In [2]:
def ip_retrieve_data():
    '''
    Pull industrial production index data from Fed
    '''
    url_base = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17'
    dates = 'from=01/01/1988&to=12/31/2019'
    series = 'series=f97ad3652d87a6c1720943c31423103a'
    settings = 'filetype=csv&label=include&layout=seriescolumn'
    
    url = f'{url_base}&{series}&lastobs=&{dates}&{settings}'
    
    raw_data = pd.read_csv(url)
    
    return raw_data

In [3]:
def ip_get_series_dict(raw_data):
    '''
    Use raw data to identify series and series names 
    and store the results as a python dictionary
    '''
    d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
         for i, v in raw_data.iloc[4, 1:].iteritems()}
    
    return d

In [4]:
def ip_clean_data(raw_data):
    '''
    Convert raw industrial production data to dataframe
    with date index column and clear column names
    '''
    date_column = raw_data.loc[5:, 'Series Description']
    date_index = pd.to_datetime(date_column).rename('Date')
    columns = [series_dict[series_id] 
               for series_id in raw_data.iloc[4, 1:].values]
    
    clean_data = raw_data.iloc[5:, 1:].astype('float')
    clean_data.index = date_index
    clean_data.columns = columns
    
    return clean_data

In [5]:
def ip_retrieve_weights(series_dict):
    '''
    Retrieve relative importance data for each series that
    was identified in the series_dict
    '''
    adj_series_dict = {k[3:-2]: v for k, v in series_dict.items()}
    series = adj_series_dict.keys()
    url = 'https://www.federalreserve.gov/releases/g17/ipdisk/'
    file = 'ipweights_sa.txt'
    columns = ['Series', 'Year', 'January', 'February', 'March', 
              'April', 'May', 'June', 'July', 'August', 
              'September', 'October', 'November', 'December']
    raw_weights = pd.read_csv(f'{url}{file}', sep='\s+', skiprows=1)
    raw_weights.columns = columns
    weights = (raw_weights[raw_weights['Series'].isin(series)]
               .set_index(['Series', 'Year']).stack().reset_index())
    weights['Date'] = (pd.to_datetime(weights['level_2'] + ' 01, ' 
                    + weights['Year'].astype('int').astype('str')))
    weights = (weights.set_index(['Series', 'Date'])[0]
               .unstack().T.rename(adj_series_dict, axis=1)
               .loc['1988':])
    
    return weights

In [6]:
def ip_growth_contrib(data, weights, series_list):
    '''
    Return the one-year contribution to industrial production growth
    for each series in the series_list. 
    
    data: dataframe with industrial production index
    weights: dataframe with relative importance of each series
    series_list: column names that match data and weight columns
    '''
    contribution = pd.DataFrame()
    for series in series_list:
        for date in data[series].index:
            series_weight = weights.loc[date, series] / 100
            growth_rate = (data[series].pct_change(12).loc[date]) * 100
            contribution.at[date, series] = growth_rate * series_weight
            
    return contribution.loc['1989':]

In [7]:
def ip_export_data(data, weights):
    '''
    Save individual csv files that feed into chartbook charts
    
    Extra steps taken to exclude quarters that aren't complete
    
    Charts:
    indpro: Total IP Index and Manufacturing Index Line Chart:
    indprogr: IP Growth Contribution by Market Group - Long-term
    indprogr_rec: IP Growth Contribution by Market Group - Recent
    indprogr2: IP Growth Contribution by Industry Group - Long-term
    indprogr_rec2: IP Growth Contribution by Industry Group - Recent
    '''
    export_file = data_dir / 'indpro.csv'
    series_list = ['Manufacturing', 'Total index']
    data[series_list].loc['1989':].to_csv(export_file, index_label='date')
    print('Saved: ', export_file)
    
    series_dict = {'indprogr': ['Consumer goods', 'ENS', 'Materials'],
                   'indprogr2': ['Durable manufacturing', 'Mining', 
                                 'Nondurable manufacturing', 
                                 'Electric and gas utilities']}
    
    for i in [data, weights]:
        i['ENS'] = i['Equipment, total'] + i['Nonindustrial supplies']
    
    quarter = data.index[-1].quarter
    year = data.index[-1].year

    full_quarter = data.index[-1]
    current_quarter_len = (
        len(data.loc[(data.index.quarter == quarter) & 
                           (data.index.year == year)]))
    
    if current_quarter_len < 3:
        full_quarter = (data.index[-1] - 
                           pd.DateOffset(months=current_quarter_len))
    
    for name, series_list in series_dict.items():
        contrib = ip_growth_contrib(data, weights, series_list)
        export_file = data_dir / f'{name}.csv'
        (contrib.resample('QS').mean().loc[:full_quarter]
         .to_csv(export_file, index_label='date'))
        print('Saved: ', export_file)
        export_file = data_dir / f'{name}_rec.csv'
        contrib.loc['2015':].to_csv(export_file, index_label='date')
        print('Saved: ', export_file)

In [28]:
def three_year_growth(data, series):
    '''Annualized growth rate over past three years'''
    return ((data[series].pct_change(36).iloc[-1] + 1)**(1/3)-1) * 100

def three_year_growth_text(clean_data, series):
    '''
    Return short text string with three year growth
    '''
    
    growth = three_year_growth(clean_data, series)
    
    if growth > 0.1:
        text = f'increased at an annual rate of {growth:.1f} percent'
    elif growth < -0.1:
        text = f'decreased at an annual rate of {abs(growth):.1f} percent'
    else:
        text = 'was virtually unchanged'
        
    return text

In [29]:
def ip_export_text(data, weights):
    '''
    Export text files that fill in text in the chartbook
    '''
    s_info = series_info(data['Manufacturing'])
    latest_date = s_info['date_latest_ft']
    mfg_3yr = three_year_growth(data, 'Manufacturing')
    tot_3yr = three_year_growth(data, 'Total index')
    max_diff = (s_info['val_latest'] / s_info['val_max'] - 1) * 100
    
    text = ('Manufacturing production increased at an average annual '+
            f'rate of {mfg_3yr:.1f} percent over the past three years, as '+
            f'of {latest_date}. The total industrial production index grew at '+
            f'an annual rate of {tot_3yr:.1f} percent over the same period. '+
            f'Manufacturing production is currently {abs(max_diff):.1f} '+
            f'percent below its peak in {s_info["date_max_ft"]}.')
    
    write_txt(text_dir / 'indpro.txt', text)
    
    print(text)
    
    cg = three_year_growth_text(data, 'Consumer goods')
    eq = three_year_growth_text(data, 'Equipment, total')
    ns = three_year_growth_text(data, 'Nonindustrial supplies')
    mat = three_year_growth_text(data, 'Materials')
    
    text = (f'By market group, production of consumer goods {cg} over the '+
            f'past three years, as of {latest_date}. Total production of '+
            f'business equipment {eq}, production of nonidustrial supplies {ns}, and '+
            f'production of materials {mat}.')
    
    write_txt(text_dir / 'indpro2.txt', text)
    
    print(text)

In [186]:
def ip_export_table(clean_data, weights):
    '''
    Create a custom chartbook table from the IP data
    '''
    export_file = data_dir / 'indpro.tex'
    
    d = {'Total index': ' & Total index',
         'Manufacturing': ' & \hspace{2mm}Manufacturing',
         'Durable manufacturing': '\cbox{blue!60!black} & \hspace{4mm}Durable manufacturing',
         'Motor vehicles and parts': ' & \hspace{6mm}Motor vehicles \& parts',
         'Nondurable manufacturing': '\cbox{blue!20!cyan!80!white} & \hspace{4mm}Nondurable manufacturing',
         'Mining': '\cbox{orange!20!yellow} & \hspace{2mm}Mining',
         'Electric and gas utilities': '\cbox{green!80!blue} & \hspace{2mm}Utilities',
         'Consumer goods': '\cbox{violet!60!black} & \hspace{2mm}Consumer goods',
         'Durable consumer goods': ' & \hspace{4mm}Consumer durables',
         'Automotive products': ' & \hspace{6mm}Automotive products',
         'Nondurable consumer goods': ' & \hspace{4mm}Consumer nondurables',
         'Foods and tobacco': ' & \hspace{6mm}Foods and tobacco',
         'Chemical products': ' & \hspace{6mm}Chemical products',
         'Consumer energy products': ' & \hspace{6mm}Consumer energy products',
         'ENS': '\cbox{magenta} & \hspace{2mm}Equipment \& nonindustrial supplies',
         'Equipment, total': ' & \hspace{4mm}Equipment',
         'Industrial equipment': ' & \hspace{6mm}Industrial equipment',
         'Nonindustrial supplies': ' & \hspace{4mm}Nonindustrial supplies',
         'Construction supplies': ' & \hspace{6mm}Construction supplies',
         'Business supplies': ' & \hspace{6mm}Business supplies',
         'Materials': '\cbox{orange!70!yellow} & \hspace{2mm}Materials',
         'Consumer parts': ' & \hspace{4mm}Consumer parts',
         'Equipment parts': ' & \hspace{4mm}Equipment parts',
         'Textile materials': ' & \hspace{4mm}Textile materials',
         'Chemical materials': ' & \hspace{4mm}Chemical materials',
         'Energy materials': ' & \hspace{4mm}Energy materials'}
    
    table = pd.DataFrame()
    data = ip_growth_contrib(clean_data, weights, d.keys())
    table = data.iloc[-3:].iloc[::-1].T
    table.columns = [date.strftime('%b %Y') for date in table.columns]
    table['1-year'] = data.rolling(12).mean().iloc[-1]
    table['3-year'] = data.rolling(36).mean().iloc[-1]
    table['10-year'] = data.rolling(120).mean().iloc[-1]
    table['30-year'] = data.rolling(360).mean().iloc[-1]
    table = table.round(2)
    table.index = [d[name] for name in table.index]

    (table.to_csv(export_file, sep='&', line_terminator='\\\ ', 
                  quotechar=' ', index_label='&'))
    
    print('Saved: ', export_file)

In [10]:
raw_data = ip_retrieve_data()

In [11]:
series_dict = ip_get_series_dict(raw_data)

In [12]:
clean_data = ip_clean_data(raw_data)

In [13]:
weights = ip_retrieve_weights(series_dict)

In [14]:
ip_export_data(clean_data, weights)

Saved:  ../chartbook/data/indpro.csv
Saved:  ../chartbook/data/indprogr.csv
Saved:  ../chartbook/data/indprogr_rec.csv
Saved:  ../chartbook/data/indprogr2.csv
Saved:  ../chartbook/data/indprogr2_rec.csv


In [30]:
ip_export_text(clean_data, weights)

Manufacturing production increased at an average annual rate of 1.0 percent over the past three years, as of October 2019. The total industrial production index grew at an annual rate of 2.1 percent over the same period. Manufacturing production is currently 5.5 percent below its peak in December 2007.
By market group, production of consumer goods increased at an annual rate of 0.3 percent over the past three years, as of October 2019. Total production of business equipment increased at an annual rate of 2.8 percent, production of nonidustrial supplies increased at an annual rate of 1.1 percent, and production of materials increased at an annual rate of 3.2 percent.


In [187]:
ip_export_table(clean_data, weights)

Saved:  ../chartbook/data/indpro.tex
